In [ ]:
""" pandas review point
1) 조인, 합성 관련 메서드
2) 통계값 관련 메서드
3) 시각화 관련 메서드
4) 전처리, 피처 엔지니어링 메서드
5) 시간 다루기, 시간 조건에 맞는 특정 유저 찾기 (0)
"""

In [ ]:
import warnings
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
""" binary classification binary_classification
resource from:
    https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023
    https://www.kaggle.com/datasets/shriyashjagtap/e-commerce-customer-for-behavior-analysis?select=ecommerce_customer_data_large.csv
"""

PATH = "binary_classification/ecommerce_customer_data_large.csv"
df = pd.read_csv(PATH)
df

In [ ]:
""" preprocess the datetime in dataframe
1) string 형태의 시간 데이터가 담긴 컬럼에 대해서 to_datetime() 적용
2) 새롭게 생성된 datetime 컬럼에서 date, time 추출
    - date: 년도, 월, 일
    - time: 시간, 분, 초
    => 리턴 타입은 string
    => 이렇게 시간 분리해두니까 깔끔해지네
    
Reference:
    https://sarah0518.tistory.com/61
"""
df["dt"] = pd.to_datetime(
    df["Purchase Date"],
    format="%Y-%m-%d %H:%M",
    errors='raise'
)
df['date'] = df['dt'].dt.date
df['year'] = df['dt'].dt.year
df['month'] = df['dt'].dt.month
df["day"] = df['dt'].dt.day
df['time'] = df['dt'].dt.time
df

In [ ]:
""" make the filtering func by using time
1) 서로 다른 날에 한 번 이상씩 구매한 사람 세기
    - 그땐 해당 아이디 리스트가 등장한 날짜를 계속 세트에 넣고, 세트 길이가 2 이상인 애들만 활성유저로 사용 했는데
    - 지금 생각해보면, 아래 조건에 맞게 월별로 리스트를 찾고, 리스트로 루프 돌리는게 맞을 듯
        - 저렇게 하면 선형 시간에 찾을 수 있는데
        - 이제 월별로 바꾸면 시간 복잡도는 O(k•N)  그래도 이중 루프는 아님! 
"""
from tqdm.auto import tqdm


# df[df['date'].astype("str") >= "2023-05-03"]["Customer ID"]
cnt_df = df[(df["year"] == 2023) & (df["month"] == 5)]
id_list = cnt_df["Customer ID"].unique()  # 이렇게 하면 넘파이 배열 나오니까 인덱싱 사용하면, 시간 복잡도를 상수 시간으로 변경


for i in range(1, 13):
    MAU = 0
    MAU_list = []
    cnt_df = df[(df["year"] == 2023) & (df["month"] == i)]
    try:
        print(cnt_df[(cnt_df["Customer ID"] == 45089) & (cnt_df["year"] == 2023) & (cnt_df["month"] == 1)])
    except:
        pass
        
    for id in tqdm(id_list):
        if len(cnt_df[cnt_df["Customer ID"] == id]['day'].unique()) > 1:
            MAU += 1
            MAU_list.append(id)

    print(f"2023.{i}'s MAU is: {MAU}")
    print(f"2023.{i}'s MAU id is: {MAU_list}")

In [ ]:
""" check the basic statistic information about dataframe """

df.info(), df.describe()

In [ ]:
""" multi-index by using set_index() """

df.set_index(['V1', 'V2'], inplace=True)  # set the multi-index
df.reset_index(inplace=True, drop=True)  # undo the multi-index

In [ ]:
""" use fillna, fillna drop """

df.dropna(subset=df.columns, inplace=True)
df

In [ ]:
""" inner join, outer join, left join, right join """


df1 = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})
df2 = pd.DataFrame({
    'ID': [3, 4, 5, 6],
    'Age': [24, 27, 22, 32]
})

# natural join: 교집합, 동시에 존재하는 키값만 결과에 포함
inner_result = pd.merge(
    df1, 
    df2,
    how="inner",
    on="ID"  # natural join
)

# outer join: 합집합
outer_result = pd.merge(
    df1,
    df2,
    how="outer",
    on="ID"
)

# left join: 왼쪽 데이터프레임 기준으로 병합
left_result = pd.merge(
    df1,
    df2,
    how="left",
    on="ID"
)

# right join: 오른쪽 데이터프레임 기준 병합
right_result = pd.merge(
    df1,
    df2,
    how="right",
    on="ID"
)

# concat: 단순 병합
# 병합 축 설정에 따라서 결과 상이
concat_result = pd.concat(
    [df1, df2],
    axis=0
)

In [ ]:
inner_result

In [ ]:
outer_result

In [ ]:
left_result

In [ ]:
right_result

In [ ]:
concat_result

In [ ]:
""" data type casting """

df['V1'] = df['V1'].astype(int)
df.info()

In [ ]:
""" use groupby """

df3 = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'C', 'C', 'A', 'B'],
    'Type': ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'],
    'Value1': [10, 20, 10, 40, 50, 60, 70, 80],
    'Value2': [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5]
})
df3

In [ ]:
df3.groupby(['Category', "Type"]).agg({
    "Value1": ["sum", "mean", "std"],
    "Value2": ["sum", "mean", "std"]
})

In [ ]:
df3.groupby(['Category']).agg({
    "Value1": ["sum", "mean", "std"],
    "Value2": ["sum", "mean", "std"]
})

In [ ]:
df3.groupby(["Category"])[["Value1", "Value2"]].sum()

In [ ]:
sum_df = df3.groupby(["Category"])[["Value1", "Value2"]]
sum_df.size()

In [ ]:
filter_df = df3.groupby(["Category"]).filter(lambda x: x.Value1.sum() >= 130)
filter_df

In [ ]:
filter_df.groupby(["Category"])[["Value1", "Value2"]].sum()

In [ ]:
df3.groupby(["Category", "Type"]).mean()

In [ ]:
df3.groupby(["Category", "Type"])[["Value1"]].sum().unstack()